In [ ]:
# default_exp executor

In [39]:
#export
from datetime import datetime
from functools import lru_cache
import uuid
import os
from pathlib import Path
import tempfile
import yaml

from blocks.filesystem import GCSFileSystem as gcsfs

In [22]:
class GCPConfig:

    @staticmethod
    @lru_cache(1)
    def bucket_name():
        return os.getenv("BUCKET_NAME") or input("Please enter the bucket name: ").strip()

    @staticmethod
    @lru_cache(1)
    def project_id():
        return os.getenv("PROJECT_ID") or input("Please enter the project id: ").strip()


In [40]:
#export
class AIP:

    @property
    @lru_cache(1)
    def job_name(self):
        now = datetime.now()
        date_time = now.strftime("%Y%m%d_%H%M%S")
        gen_uunid = hex(uuid.getnode())
        return f"ai_run_{date_time}_{gen_uunid}"

    @property
    @lru_cache(1)
    def job_dir(self):
        return ""

    def run(self, output_dir: Path, python_version: str = "3.7", runtime_version: str = "2.1", scale_tier: str = "CUSTOM", region: str = "us-central1", machine_type: str = "n1-highmem-32"):
        job_name = self.job_name
        trainer_package_path = output_dir / "src"
        job_dir = output_dir / "output"

        # prepare the folder on gcp

        # upload to gcp
        main_trainer_module = output_dir / "src" / "CHANGE_THIS"

        # config
        config = {
            "trainingInput":
            {
                "scaleTier": "CUSTOM",
                "masterType": machine_type
            }
        }
        with tempfile.NamedTemporaryFile(mode = "w") as fp:
            yaml.dump(config, fp, sort_keys=True)
            config_path = fp.name

            # run the job
            gcp_cmd = f"""
                gcloud ai-platform jobs submit training {job_name} \
                --job-dir {job_dir} \
                --package-path {trainer_package_path} \
                --module-name {main_trainer_module} \
                --region {region} \
                --master-image-uri {image}
                --runtime-version={runtime_version} \
                --python-version={python_version} \
                --scale-tier {scale_tier} \
                --config {config_path}
            """
            print(gcp_cmd)

In [41]:
AIP().run(Path("gs://testjobsubmit"))


                gcloud ai-platform jobs submit training ai_run_20200428_083645_0x8c8590a5b94c                 --job-dir gs:/testjobsubmit/output                 --package-path gs:/testjobsubmit/src                 --module-name gs:/testjobsubmit/src/CHANGE_THIS                 --region us-central1                 --runtime-version=2.1                 --python-version=3.7                 --scale-tier CUSTOM                 --config /var/folders/2k/b58ly_192yjgtv76zjxqj6f8_9cn2g/T/tmpl8pefm4j
            
